## About Caida_0.dat
- Distinct element: 177335
- Number of incomming element: 2472727

### 衡量指標公式
![Alt text](https://ithelp.ithome.com.tw/upload/images/20191106/20001976lmUB0yA0ju.png)

In [3]:
# count distinct element frequency
import pandas as pd
import os 

# datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
# datali=os.listdir(datapath)[0]
src_data='caida_0.dat'

num_line=0
count=0
itemdict={}

with open(src_data, 'rb') as file:
    while True:
        line=str(file.read(13))
        if len(line)<13:
            break
        else:
            num_line+=1         
            if line not in itemdict:
                itemdict[line]=1
            else:
                itemdict[line]+=1

df=pd.DataFrame(list(itemdict.items()),columns=['Element', 'Count'])
df=df.reset_index(drop=True)
df=df.sort_values('Count',ascending=False)

print("Distinct element: {}".format(len(itemdict)))
print("Number of incomming element: {}".format(num_line))
print(df.head(50))

df.to_csv("caida_0_distinct_count.csv",index=False)


Distinct element: 177335
Number of incomming element: 2472727
                                                  Element  Count
58            b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06'  16889
274        b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06'  11647
706                 b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06'  10113
6877       b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06'   8993
1227         b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06'   8418
102824           b'\x99\xc1\xe9Z\x00P+\xfc\xe0=\xe7%\x06'   6594
31291   b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06'   6244
1161       b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06'   5420
2965          b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06'   5194
1662       b'E\x0c\xd5\xf5\xa4\x14\xdd.\xdc\xec\x00P\x06'   5044
2050       b'\x01l$\xce\x0e\xad\x01l\xc5\xc6\x01\xbb\x06'   4444
1036       b'\xdd\x01b\xdf\xd5\xce\xdd.\xdc\xf1\x00P\x06'   4246
11380         b'~.~\x13\xa5\xb3+\xfc\xe2\x05\x01\xbb\x06'   4081
242       b'\r\xe1\x94\xb7\x

## Space Saving
- Top-1024 with size 9024 bytes.
- Execution time:746.1087837219238 seconds.
- Find:327,TP:13,FP:314

In [ ]:
# space saving caida_0, StreamSummary with 1024 elements, full test
# built-in list implement
# Top-1024 with size 9328 bytes.
# Execution time:581.9553816318512 seconds.


import sys
import os
import time
start=time.time()

datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
datali=os.listdir(datapath)[0]

size=1024
itemlist=[]
with open(os.path.join(datapath,datali), 'rb') as file:
    while True:
        line=str(file.read(13))
        if len(line)<13:
            print('EOF')
            break
        else:
            if len(itemlist)==0:
                itemlist.append([line,1])
            else:
                for i in range(len(itemlist)):
                    if line==itemlist[i][0]:
                        itemlist[i][1]+=1
                        found=True
                        #print("found in itemlist[{}]".format(i))
                        break
                    else:
                        found=False
                if found:
                    pass
                else:
                    if len(itemlist)<size:
                        itemlist.append([line,1])
                    else:
                        # len(itemset)=size
                        last=len(itemlist)-1
                        itemlist[last][0]=line
                        itemlist[last][1]+=1
            itemlist=sorted(itemlist,key = lambda itemlist: itemlist[1],reverse=True)
        #print("itemlist:{}\nlen(itemlist):{}\n".format(itemlist,len(itemlist)))
end=time.time()

print(itemlist[:50],len(itemlist))
print("Top-{} with size {} bytes.".format(len(itemlist),sys.getsizeof(itemlist)))
print("Execution time:{} seconds.".format(str(end-start)))

In [5]:
# space saving caida_0, StreamSummary with 1024 elements, full test
# class implement


import sys
import time
import operator

class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self):
        super().__init__()
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return 'total count: {}, distinct element: {}'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return 'ID: {}, count: {}'.format(self.ID,self.count)
    def __repr__(self):
        return "'{}', count: {}".format(self.ID,self.count)

start=time.time()
'''
datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
datali=os.listdir(datapath)[0]
'''
src_data='caida_0.dat'
size=1024
Top=[]
item_count=0
# Space-Saving

with open(src_data,'rb') as file:
    while True:
        element=str(file.read(13))
        if len(element)<13:
            print('EOF')
            break
        else:
            item_count+=1
            # print("read {}th element: {}".format(item_count,element))
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count()
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        Top[-1].ID=element
                        Top[-1].add_count()
                # Top= sorted(Top, key=operator.attrgetter('count'),reverse=True)
                # non-inplace sort
                Top.sort(key=operator.attrgetter('count'),reverse=True)
                    # inplce sort
end=time.time()
print(Top[:50],len(Top))
print("Top-{} with size {} bytes.".format(len(Top),sys.getsizeof(Top)))
print("Execution time:{} seconds.".format(str(end-start)))

#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("SpaceSaving_caida_0_distinct_count.csv",index=False)
df.head(50)

EOF
['b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06'', count: 16889, 'b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06'', count: 11647, 'b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06'', count: 10113, 'b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06'', count: 9066, 'b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06'', count: 8424, 'b'\x99\xc1\xe9Z\x00P+\xfc\xe0=\xe7%\x06'', count: 7563, 'b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06'', count: 6365, 'b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06'', count: 5421, 'b'\xd2\x1e\xbeu\x01\xbb+\xfc\xe0.\x92\xd5\x06'', count: 5213, 'b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06'', count: 5198, 'b'E\x0c\xd5\xf5\xa4\x14\xdd.\xdc\xec\x00P\x06'', count: 5077, 'b'\x01l$\xce\x0e\xad\x01l\xc5\xc6\x01\xbb\x06'', count: 4446, 'b'\xdd\x01b\xdf\xd5\xce\xdd.\xdc\xf1\x00P\x06'', count: 4247, 'b'OZ\x0e\xae\xdc\xf1o\xcd\x19\xf0\x1f\x90\x11'', count: 4225, 'b'~.~\x13\xa5\xb3+\xfc\xe2\x05\x01\xbb\x06'', count: 4134, 'b'\r\xe1\x94\xb7\xc2>\x01`\xdf\x92\x01\xbb\x06'', count: 4081, 'b'\xc

,ID,Count
0,b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06',16889
1,b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06',11647
2,b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06',10113
3,b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06',9066
4,b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06',8424
5,b'\x99\xc1\xe9Z\x00P+\xfc\xe0=\xe7%\x06',7563
6,b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06',6365
7,b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06',5421
8,b'\xd2\x1e\xbeu\x01\xbb+\xfc\xe0.\x92\xd5\x06',5213
9,b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06',5198


In [17]:
# SpaceSaving result compare
import pandas as pd
grtruth=pd.read_csv('caida_0_distinct_count.csv')
SS_result=pd.read_csv('SpaceSaving_caida_0_distinct_count.csv')

gli = grtruth.values.tolist()
mli=SS_result.values.tolist()

tp=0
fp=0
find=0
err=[]

for item in mli:
    for element in gli[:1024]:
        if item[0]==element[0]:
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                err.append([item[0],element[1]-item[1]])
                fp+=1
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
# print(err)
                

Find:327,TP:13,FP:314


## Count min sketch
- Total memory 4824 bytes :Top-512 with size 4720 bytes+ CMS with size 104 bytes.
- Execution time:673.5203354358673 seconds.

- Total memory 4824 bytes :Top-512 with size 4720 bytes+ CMS with size 104 bytes.
- Execution time:1999.0816707611084 seconds.

In [ ]:
# count-min sketch webdocs, top-512, full read
import sys
import os
import time
from probables import (CountMinSketch)
import pandas as pd

start=time.time()
topk=[]
size=1024
item_count=0
cms = CountMinSketch(width=1024, depth=4)
'''
datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
datali=os.listdir(datapath)[0]
'''
src_data='caida_0.dat'


# if os.path.exists(os.path.join(datapath,datali)):
if os.path.exists(src_data):
    with open(src_data,'rb') as file:
        while True:
            line=str(file.read(13))
            if len(line)<13:
                print('EOF')
                break
            else:
                item_count+=1
                # print("read {}th element: {}".format(item_count,element))
                cms.add(line)
                count=cms.check(line)
                if len(topk)==0:
                    topk.append([line,count])
                else:
                    find=False
                    for i in range(len(topk)):
                        if line==topk[i][0]:
                            topk[i][1]=count
                            find=True
                            break
                    if find==False:
                        if len(topk)<size:
                            topk.append([line,count])
                        else:
                            topk.append([line,count])
                            topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                            topk.pop()
                topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    
    end=time.time()
    print(topk[:50],len(topk))
    print("Total memory {3} bytes :Top-{0} with size {1} bytes+ CMS with size {2} bytes.".format(len(topk),sys.getsizeof(topk),sys.getsizeof(cms),sys.getsizeof(cms)+sys.getsizeof(topk)))
    print("Execution time:{} seconds.".format(str(end-start)))
else:
    print("file doesn't exist")

#　conver Top into df    
templi=[]
for i in topk:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("CM_caida_0_distinct_count.csv",index=False)
df.head(50)

In [ ]:
# CM result compare
import pandas as pd
grtruth=pd.read_csv('caida_0_distinct_count.csv')

cm_result=pd.read_csv('CM_caida_0_distinct_count.csv')

gli = grtruth.values.tolist()
cli=cm_result.values.tolist()

tp=0
fp=0
find=0
err=[]

for item in cli:
    for element in cli[:1024]:
        if item[0]==element[0]:
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                err.append([item[0],element[1]-item[1]])
                fp+=1
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
print(err)

## My algo
- Total memory 4368 bytes.
- Execution time: 470.3432066440582 seconds

- Top-512 + SK-1024*4
    - Total memory 4368 bytes.
    - Execution time: 1621.5535824298859 seconds
- Top-1024 + SK-1024*4    
    - Total memory 9120 bytes.
    - Execution time: 1535.7936234474182 seconds   
    - Find:476,TP:112,FP:364

In [14]:
# Caida 2016
# 只讀1個檔(0.dat)
# Execution time: 582.126701593399 seconds

import sys
import os
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import pandas as pd
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    #　width+=1
    e_max=get_emax()
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    item=Tail(ID,element.count)
    
    # print('UpdateSk:{}->ID {}'.format(element,ID))
    # print("hash1: {}".format(hash1))
    # print("hash2: {}".format(hash2))
    # print("match row: {}".format(row))
    # print("SK[{}]:{}".format(row,Sk[row]))
    # print("len(Sk[{}]:{})".format(row,len(Sk[row])))
    
    Sk[row][0].add_count(item.count)
        # total count+=count
    if len(Sk[row])==1:
        # Head only, append e directly
        Sk[row].append(item)
        Sk[row][0].maxID=element.ID
        match=True
        index=1
            # e is in Sk[row][1]
        # print('Sk[{}] when len(Sk[{}])==1:\n\t{}'.format(row,row,Sk[row]))
        # print("After append:\n{}".format(Sk[row]))
        # print("length of Sk[row]: {}".format(len(Sk[row])))
    elif len(Sk[row])<width:
        put_in=random.choice([0,1],size=1,p=[len(Sk[row])/width,1-len(Sk[row])/width])[0]
            # send into SK[row] with prob. 1-len(SK[row])/width
        for i in range(1,len(Sk[row])):
            # checks whether e is in SK[row]
            if ID == Sk[row][i].ID:
                match=True
                Sk[row][i].add_count(item.count)
                index=i
                '''
                print('{} matches in Sk[{}][{}]:{}'.format(item.ID,row,index,Sk[row][index].ID))
                print('Sk[{}]:\n\t{}'.format(row,Sk[row]))
                '''
                break
            else:
                match=False
        if not match:
            if put_in==1:
                # send e into SK[row]
                # print('put_in={}, send e into SK[{}]'.format(put_in,row))
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
                # print('Sk[{}]:\n\t{}\n'.format(row,Sk[row]))
            else:
                Sk[row][0].distinct.add(ID)
                ''' 
                print('put_in={}, send e into other,update distinct'.format(put_in,len(Sk[row][0].distinct)))
                print('Sk[{}]:\n\t {}'.format(row,Sk[row]))
                '''
    elif len(Sk[row])==width:
        # len(Sk[row])=width, put e into other
        Sk[row][0].distinct.add(ID)
   
    if match:
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
    else:
        # avg count of other > e_max.count
        avg=(Sk[row][0].count-sum(Sk[row][i].count for i in range(len(Sk[row]))))/len(Sk[row][0].distinct)
        if avg >e_max.count:
            e_max=Tail(element.ID,Sk[row][index].count)
# ========================== Update Sketch==========================
def get_emax():
    return e_max
# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
def DeleteSk(element):
    # 刪除e_max，找新的e_max
    # print('Delete start======================================')
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    index=0
    for i in range(1,len(Sk[row])):
        if Sk[row][i].ID==ID:
            # print("found {} at SK[{}][{}], ID={}".format(element.ID,row,i,Sk[row][i].ID))
            index=i
            # print("pop element:{}".format(Sk[row][index]))
            Sk[row].pop(index)
            break
    # print('Delete Sk for find max of Sk[i] len(Sk):{}'.format(len(Sk)))
    Sk[row][0].count-=element.count
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    print('Delete Sketch over======================================')
    '''

# ========================== BringBack=========================

start=time.time()
'''
datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
datali=os.listdir(datapath)[0]
'''
src_data='caida_0.dat'

depth=4
width=128
size=1024
Top=[]
Sk=[[Head(0)] for j in range(depth)]
e_max=Tail('',0)
item_count=0


# os.path.join(datapath,datali),'rb'
with open(src_data,'rb') as file:
    # print("file open now: {}".format(file.name))
     #以binary讀取，資料型態也為byte
    while True:
        element=str(file.read(13))
        if len(element)<13:
            print('EOF')
            break
        else:
            item_count+=1
            #　print("read {}th element: {}".format(item_count,element))
            # Update_Top(Tail(element,1))
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count(1)
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        # print("e_max in main, id(e_max):{}".format(id(e_max)))
                        UpdateSk(Tail(element,1),width,depth)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            # print("Top:{}".format(Top))
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # bring back
                # 要去掉SK[row]中的e_max.ID及count的node
                # print('Top after BringBack: \n\t{}'.format(Top))

    end=time.time()
    print("Top[50]:\n\t{}".format(Top[:50]))
    print("Sk:\n\t{}".format(Sk))
    print("Total memory {} bytes.".format(sys.getsizeof(Top)+sys.getsizeof(Sk)))
    print('Execution time: {} seconds'.format(str(end-start)))

    
#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("My_algo_1024_caida_0_distinct_count.csv",index=False)
df.head(50)



EOF
Top[50]:
	[[ID: b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06', count: 16889], [ID: b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06', count: 11647], [ID: b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06', count: 10113], [ID: b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06', count: 9010], [ID: b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06', count: 8418], [ID: b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06', count: 6264], [ID: b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06', count: 5421], [ID: b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06', count: 5194], [ID: b'E\x0c\xd5\xf5\xa4\x14\xdd.\xdc\xec\x00P\x06', count: 4991], [ID: b'\x01l$\xce\x0e\xad\x01l\xc5\xc6\x01\xbb\x06', count: 4444], [ID: b'\xdd\x01b\xdf\xd5\xce\xdd.\xdc\xf1\x00P\x06', count: 4246], [ID: b'~.~\x13\xa5\xb3+\xfc\xe2\x05\x01\xbb\x06', count: 4089], [ID: b'\r\xe1\x94\xb7\xc2>\x01`\xdf\x92\x01\xbb\x06', count: 4081], [ID: b'\xcf\x88\xee\xa0\x17\x01\x01`\xa6\xcc\x01\xbb\x06', count: 4053], [ID: b'\x800\xeeP <\x01`\xa6\xc7\x01\xbb\x06', count: 

,ID,Count
0,b'\xd2\x1d9\xfa\x00P\x0f\x13I\x9a\xf89\x06',16889
1,b'E.\x1c)\xa4\xb2\x89\xb6\x06\xba\xd2\xad\x06',11647
2,b'V+x\xe8\x00Po\xcd\xe4\xde\xccw\x06',10113
3,b'\xb7\xf6h\xd0\xc1\x8b+\xfc\xe1\x08\x00P\x06',9010
4,b'\x01l|\xff\x8f\r\x01l\xc5\xc8\x01\xbb\x06',8418
5,b'c\x0f\xb2\xd4\x80\xd2\x01`\x8d\x12\x01\xbb\x06',6264
6,b'\x01l|\xec\xcc\x1b\x01l\xc5\xc8\x01\xbb\x06',5421
7,b'E\x0c\xd5\xf7\xd7(\xdd.\xdc\xec\x00P\x06',5194
8,b'E\x0c\xd5\xf5\xa4\x14\xdd.\xdc\xec\x00P\x06',4991
9,b'\x01l$\xce\x0e\xad\x01l\xc5\xc6\x01\xbb\x06',4444


In [16]:
# My algorithm result compare
import pandas as pd
grtruth=pd.read_csv('caida_0_distinct_count.csv')

My_result=pd.read_csv('My_algo_1024_caida_0_distinct_count.csv')

gli = grtruth.values.tolist()
mli=My_result.values.tolist()

tp=0
fp=0
find=0
err=[]

for item in mli:
    for element in gli[:1024]:
        if item[0]==element[0]:
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                err.append([item[0],element[1]-item[1]])
                fp+=1
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
print(err)
                

Find:476,TP:112,FP:364
[["b'\\xb7\\xf6h\\xd0\\xc1\\x8b+\\xfc\\xe1\\x08\\x00P\\x06'", -17], ["b'c\\x0f\\xb2\\xd4\\x80\\xd2\\x01`\\x8d\\x12\\x01\\xbb\\x06'", -20], ["b'\\x01l|\\xec\\xcc\\x1b\\x01l\\xc5\\xc8\\x01\\xbb\\x06'", -1], ["b'E\\x0c\\xd5\\xf5\\xa4\\x14\\xdd.\\xdc\\xec\\x00P\\x06'", 53], ["b'~.~\\x13\\xa5\\xb3+\\xfc\\xe2\\x05\\x01\\xbb\\x06'", -8], ["b'\\xcf\\x88\\xee\\xa0\\x17\\x01\\x01`\\xa6\\xcc\\x01\\xbb\\x06'", 1], ["b'\\x800\\xeeP <\\x01`\\xa6\\xc7\\x01\\xbb\\x06'", 70], ["b'\\x82+!!\\xa5s\\x01`\\x92\\xe5\\x01\\xbb\\x06'", -3], ["b'ur=\\xf2\\xc1@+\\xfc\\xe2\\x06\\x00P\\x06'", -11], ["b'\\x80-\\xe5dw\\xd1\\x01`\\xa6\\xec\\x01\\xbb\\x06'", 1], ["b'\\xd0\\xfd=\\x0e\\xad\\x92\\xdd\\xc3\\xe7#\\x00\\x14\\x06'", 5], ["b'w\\xfa\\x1d\\xcf\\x86_\\x01`\\xa6\\xd5\\x01\\xbb\\x06'", -3], ["b'd\\xa2\\x1d<\\xdc\\x14\\x9d\\x0e\\x1d\\xb2\\x1fI\\x06'", -3], ["b'\\xdd\\x01b\\xdd\\xdd\\xcb\\xdd.\\xdc\\xef\\x00P\\x06'", 6], ["b'l*\\xf2{qe\\x01`\\xa7\\x11\\x01\\xbb\\x06'", -1], ["b'\\xb7\\x96U\\xe

In [ ]:
# element estimation

x="b'\x99\xc1\xe9Z\x00P+\xfc\xe0=\xe7%\x06'"
depth=4
width=128
hash1=spookyhash.hash32(bytes(x,encoding='utf-8'))
hash2=mmh3.hash(str(hash1), signed=False)
row=hash1 % depth
ID=hash2 % ((width*3)//2)
print(row,ID)
for i in Sk[row]:
    if i.ID==ID:
        print(i.count)
    else:
        print("Not found")


In [9]:
# 備用，未修改
import sys
import os
import time
import spookyhash
import mmh3
import hyperloglog
import operator
import pandas as pd
from numpy import random

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

# ========================== Update Sketch==========================
def UpdateSk(element,width,depth):
    #　width+=1
    e_max=get_emax()
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    item=Tail(ID,element.count)
    
    # print('UpdateSk:{}->ID {}'.format(element,ID))
    # print("hash1: {}".format(hash1))
    # print("hash2: {}".format(hash2))
    # print("match row: {}".format(row))
    # print("SK[{}]:{}".format(row,Sk[row]))
    # print("len(Sk[{}]:{})".format(row,len(Sk[row])))
    
    Sk[row][0].add_count(item.count)
        # total count+=count
    if len(Sk[row])==1:
        # Head only, append e directly
        Sk[row].append(item)
        Sk[row][0].maxID=element.ID
        match=True
        index=1
            # e is in Sk[row][1]
        # print('Sk[{}] when len(Sk[{}])==1:\n\t{}'.format(row,row,Sk[row]))
        # print("After append:\n{}".format(Sk[row]))
        # print("length of Sk[row]: {}".format(len(Sk[row])))
    elif len(Sk[row])<width:
        put_in=random.choice([0,1],size=1,p=[len(Sk[row])/width,1-len(Sk[row])/width])[0]
            # send into SK[row] with prob. 1-len(SK[row])/width
        for i in range(1,len(Sk[row])):
            # checks whether e is in SK[row]
            if ID == Sk[row][i].ID:
                match=True
                Sk[row][i].add_count(item.count)
                index=i
                '''
                print('{} matches in Sk[{}][{}]:{}'.format(item.ID,row,index,Sk[row][index].ID))
                print('Sk[{}]:\n\t{}'.format(row,Sk[row]))
                '''
                break
            else:
                match=False
        if not match:
            if put_in==1:
                # send e into SK[row]
                # print('put_in={}, send e into SK[{}]'.format(put_in,row))
                Sk[row].append(item)
                match=True
                index=len(Sk[row])-1
                # print('Sk[{}]:\n\t{}\n'.format(row,Sk[row]))
            else:
                Sk[row][0].distinct.add(ID)
                ''' 
                print('put_in={}, send e into other,update distinct'.format(put_in,len(Sk[row][0].distinct)))
                print('Sk[{}]:\n\t {}'.format(row,Sk[row]))
                '''
    elif len(Sk[row])==width:
        # len(Sk[row])=width, put e into other
        Sk[row][0].distinct.add(ID)
   
    if match:
        if Sk[row][index].count>e_max.count:
            e_max.ID=element.ID
            e_max.count=Sk[row][index].count
    else:
        # avg count of other > e_max.count
        avg=(Sk[row][0].count-sum(Sk[row][i].count for i in range(len(Sk[row]))))/len(Sk[row][0].distinct)
        if avg >e_max.count:
            e_max=Tail(element.ID,Sk[row][index].count)
# ========================== Update Sketch==========================
def get_emax():
    return e_max
# ========================== BringBack=========================
def BringBack(e_min,e_max):
    # print("Top before Bringback:\n\t{}".format(Top))
    e_max=get_emax()
    # print('e_max at first Bringback:{},id(e_max):{}'.format(e_max,id(e_max)))
    temp=Tail(e_min.ID,e_min.count)
    Top[-1].ID=e_max.ID
    Top[-1].count=e_max.count
    Top.sort(key=operator.attrgetter('count'),reverse=True)
    # print('Top after BringBack:\n\t{}'.format(Top))
    DeleteSk(e_max)
        # e_max in Sk[row]
    e_max.ID=''
    e_max.count=0
    # print("e_max after delete:{},id(e_max):{}".format(e_max,id(e_max)))
    UpdateSk(temp,width,depth)
    
    # print("Sk[] after Update {}:\n\t{}".format(e_min,Sk))

# ========================== BringBack=========================
def DeleteSk(element):
    # 刪除e_max，找新的e_max
    # print('Delete start======================================')
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    index=0
    for i in range(1,len(Sk[row])):
        if Sk[row][i].ID==ID:
            # print("found {} at SK[{}][{}], ID={}".format(element.ID,row,i,Sk[row][i].ID))
            index=i
            # print("pop element:{}".format(Sk[row][index]))
            Sk[row].pop(index)
            break
    # print('Delete Sk for find max of Sk[i] len(Sk):{}'.format(len(Sk)))
    Sk[row][0].count-=element.count
    '''
    print("index:{}".format(index))
    print('After pop:')
    print("\tSk[{}]={}".format(row,Sk[row]))
    print("Top:\n\t{}".format(Top))
    print('Delete Sketch over======================================')
    '''

# ========================== BringBack=========================

start=time.time()
'''
datapath=r'X:\\NTU\\ML-sketch\\dataset\\caida2016\\'
datali=os.listdir(datapath)[0]
'''
src_data='caida_0.dat'

depth=4
width=128
size=1024
Top=[]
Sk=[[Head(0)] for j in range(depth)]
e_max=Tail('',0)
item_count=0


# os.path.join(datapath,datali),'rb'
with open(src_data,'rb') as file:
    # print("file open now: {}".format(file.name))
     #以binary讀取，資料型態也為byte
    while True:
        element=str(file.read(13))
        if len(element)<13:
            print('EOF')
            break
        else:
            item_count+=1
            #　print("read {}th element: {}".format(item_count,element))
            # Update_Top(Tail(element,1))
            if len(Top)==0:
                Top.append(Tail(element,1))
            else:
                found=False
                for i in range(len(Top)):
                    if Top[i].ID==element:
                        found=True
                        Top[i].add_count(1)
                        break
                if not found:
                    if len(Top)<size:
                        Top.append(Tail(element,1))
                    else:
                        # print("e_max in main, id(e_max):{}".format(id(e_max)))
                        UpdateSk(Tail(element,1),width,depth)
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            # print("Top:{}".format(Top))
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # bring back
                # 要去掉SK[row]中的e_max.ID及count的node
                # print('Top after BringBack: \n\t{}'.format(Top))

    end=time.time()
    print("Top[50]:\n\t{}".format(Top[:50]))
    print("Sk:\n\t{}".format(Sk))
    print("Total memory {} bytes.".format(sys.getsizeof(Top)+sys.getsizeof(Sk)))
    print('Execution time: {} seconds'.format(str(end-start)))

    
#　conver Top into df    
templi=[]
for i in Top:
    templi.append([i.ID,i.count])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
df.to_csv("My_algo_1024_caida_0_distinct_count.csv",index=False)
df.head(50)



True